<a href="https://colab.research.google.com/github/dnguyend/lagrange_rayleigh/blob/master/EigenTensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\newcommand{\by}{\boldsymbol{y}}$
$\newcommand{\bu}{\boldsymbol{u}}$
$\newcommand{\bz}{\boldsymbol{z}}$
$\newcommand{\bx}{\boldsymbol{x}}$
$\newcommand{\bg}{\boldsymbol{g}}$
$\newcommand{\bH}{\boldsymbol{H}}$
$\newcommand{\bI}{\boldsymbol{I}}$
$\newcommand{\bU}{\boldsymbol{U}}$
$\newcommand{\bT}{\boldsymbol{T}}$
$\newcommand{\bF}{\boldsymbol{F}}$
$\newcommand{\bJ}{\boldsymbol{J}}$
$\newcommand{\bA}{\boldsymbol{A}}$
$\newcommand{\NCM}{\text{NCM}}$
$\newcommand{\ONCM}{\text{O-NCM}}$
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dnguyend/lagrange_rayleigh/blob/master/EigenTensor.ipynb)
# Schur form of Lagrange-Rayleigh algorithm for Eigen-Tensor problem.

A good example to compare the Schur versus Riemnannian form of the Lagrange-Rayleigh algorithm is the eigen-tensor problem.
The paper [1] proposed two methods , $\NCM$ (Newton correction method) and $\ONCM$ (orthogonal $\NCM$). We can see $\ONCM$ is Riemannian-Newton on the sphere: the updating vector $\bu$ is on the tangent space, and the updating equation is the Riemannian-Newton equation. The updating equation for NCM requires solving $\bH \by =  \bg$ where $\bH$ is of form $\nabla^2 -\lambda\bI$
which is an extension of the resolvent equation. It converges quadratically, but the increment $\by$ is not on the tangent plane. Our Schur form Rayleigh formulation provides an updating $\by_1= \by -c \zeta$ that is on the tangent space and could be computed by solving $\bH$. $\ONCM$ on the other hand, requires solving $\bH_p \bz = -\bU'\bg$, the projected Hessian ($\bU'$ is the projection). While $\bH_p$ has dimension one less than $\bH$, we need extra steps to compute the projected Hessian, as well as the imbedding of $\bz$ from the tangent space back to $\bu$ in the ambient space. [1] found $\NCM$ is inferior to $\ONCM$. Our Schur form is competitive in this case, as we show below that it generally provides around 30% in improvement in execution time over our python implementation of $\ONCM$. In theorem $\by_1$ should be identical to $\bu$ of $\ONCM$, as we proved that Schur form is just another way of solving the updating equation. It is indeed so most of the time, however there are instances where numerical discrepencies makes the two iterations diverges. However, Schur form solution is also quite stable and given the time improvement should be a competitive candidate to solving eigen-tensor problems.

[1] *Newton Correction Methods for Computing Real Eigenpairs of Symmetric Tensors.*
Ariel Jaffe, Roi Weiss, and Boaz Nadler
SIAM Journal on Matrix Analysis and Applications 2018 39:3, 1071-1094

First we pull the code from our github. Since we do not have matlab, we quickly convert the matlab code for [1] from https://github.com/arJaffe/BinaryLatentVariables/tree/master/NCM_functions to python.

We also implement a custom copy of Schur form solution. The code for these two functions are in lagrange_rayleigh/core/eigen_tensor_solver. The readers can run
!cat lagrange_rayleigh/core/eigen_tensor_solver.py
to view the code.

In [0]:
!git clone https://github.com/dnguyend/lagrange_rayleigh

fatal: destination path 'lagrange_rayleigh' already exists and is not an empty directory.


In [0]:
# run this cell to view the codes of the two routines, one for ONCM and one for
# Schur-Rayleigh:
!cat lagrange_rayleigh/core/eigen_tensor_solver.py

We also show how to use our library routine rayleigh_quotient_iteration and rayleigh_chebyshev, by first derive from the class Lagrangian and providing it with methods to compute the function to solve and derivatives. In this case the function is given by $\bF(\bx) = \bT(\bI, \bx,\cdots\bx)$. Its derivatives is $\bJ_{\bF}=\bT(\bI,\bI,\bx,\cdots \bx)$. We also compute the second derivative of $\bF$ for Rayleigh-Chebyshev.

In [0]:
from __future__ import print_function
import numpy as np
import pandas as pd
import time


import lagrange_rayleigh.core.utils as utils
from lagrange_rayleigh.core.vector_lagrangian import explicit_vector_lagrangian
from lagrange_rayleigh.core.constraints import base_constraints

from lagrange_rayleigh.core.solver import rayleigh_quotient_iteration
from lagrange_rayleigh.core.solver import rayleigh_chebyshev
from lagrange_rayleigh.core.eigen_tensor_solver import\
    orthogonal_newton_correction_method, schur_form_rayleigh,\
    symmetric_tv_mode_product


class eigen_tensor_lagrange(explicit_vector_lagrangian):
    def calc_H(self, x):
        return x[:, None]

    def F(self, x):
        v = self._args['A'].copy()
        for i in range(self._m-3):
            v = np.tensordot(v, x, axes=1)
        self._F2 = v
        self._F1 = np.tensordot(self._F2, x, axes=1)
        self._F0 = np.tensordot(self._F1, x, axes=1)
        self._F1 *= (self._m - 1)
        self._F2 *= (self._m - 2) * (self._m - 1)
        return self._F0
    
    def __init__(self, A):
        self._args = {'A': A}
        self._k = A.shape[0]
        self._m = len(A.shape)
        self._shape_in = (A.shape[0], 0)
        self._shape_out = (A.shape[0], 0)

    def calc_J_F(self, x):
        return self._F1

    def calc_J_H(self, x):
        return np.eye(x.shape[0]).reshape(
            x.shape[0], 1, x.shape[0])

    def J_F2(self, d_x):
        return np.tensordot(
            np.tensordot(self._F2, d_x, axes=1), d_x, axes=1)

    def J_C(self, d_x):
        return np.dot(
            self._state['J_C'], d_x)

    def calc_J_F2(self, x):
        return self._F2

    def calc_J_H2(self, x):
        pass

    def J_H2(self, d_x, d_lbd):
        return np.zeros((d_x.shape[0]))

    def J_C2(self, d_x):
        return 2 * np.dot(d_x.T, d_x).reshape(1)
    
    def calc_J_RAYLEIGH(self, x):
        return (- 2 * self['RAYLEIGH'] * x.T +
                self._F0.T + np.dot(x.T, self._F1)).reshape(1, -1)

Our next function calls and compare four routines, orthogonal_newton_correction_method, schur_form_rayleigh, rayleigh_quotient_iteration and rayleigh_chebyshev. We note schur_form_rayleigh and rayleigh_quotient_iteration are just different implementations of the same algorithm, the later means to be a general purpose routine so not quite efficient. schur_form_rayleigh is modelled on the style of orthogonal_newton_correction_method where we just replace solving the projected Hessian by the Hessian equation and apply the Schur form adjustment. However, the first three routines still show discrepancies, as for a small number of initial values they converge to different eigenpairs, this is due to numerical errors difficult to pinpoint.

In [0]:
def test_eigen_tensor(k, m, max_err, max_itr, n_test):
    def sphere_func(x):
        return np.dot(x.T, x) - 1

    def sphere_jacobian(x):
        return 2 * x.reshape(1, -1)

    def sphere_retraction(x, u):
        return (x + u) / np.linalg.norm(x + u)
    
    sphere = base_constraints(
        shape_in=(k,),
        shape_constraint=(1,),
        equality=sphere_func)

    sphere.set_analytics(
        J_C=sphere_jacobian,
        retraction=sphere_retraction)

    A = utils.generate_symmetric_tensor(k, m)
    e = eigen_tensor_lagrange(A)
    e.constraints = sphere

    o_ncm_cnt = np.zeros(n_test, dtype=int)
    schur_cnt = np.zeros(n_test, dtype=int)
    ray_cnt = np.zeros(n_test, dtype=int)
    ray_cheb_cnt = np.zeros(n_test, dtype=int)

    o_ncm_err = np.zeros(n_test)
    schur_err = np.zeros(n_test)
    ray_err = np.zeros(n_test)
    ray_cheb_err = np.zeros(n_test)

    o_ncm_lbd = np.zeros(n_test)
    schur_lbd = np.zeros(n_test)
    ray_lbd = np.zeros(n_test)
    ray_cheb_lbd = np.zeros(n_test)

    o_ncm_time = np.zeros(n_test)
    schur_time = np.zeros(n_test)
    ray_time = np.zeros(n_test)
    ray_cheb_time = np.zeros(n_test)
    
    for jj in range(n_test):
        x0 = np.random.randn(k)
        x0 = x0 / np.linalg.norm(x0)

        # do orthogonal
        t_start = time.time()
        o_x, o_lbd, ctr, converge = orthogonal_newton_correction_method(
            A, max_itr, max_err, x_init=x0)
        t_end = time.time()
        o_ncm_cnt[jj] = ctr
        o_ncm_lbd[jj] = o_lbd
        o_ncm_err[jj] = np.linalg.norm(
            symmetric_tv_mode_product(
                A, o_x, m-1) - o_lbd * o_x)
        o_ncm_time[jj] = t_end - t_start

        # do schur_form_rayleigh
        t_start = time.time()
        s_x, s_lbd, ctr, converge = schur_form_rayleigh(
            A, max_itr, max_err, x_init=x0)
        t_end = time.time()
        schur_cnt[jj] = ctr
        schur_lbd[jj] = s_lbd
        schur_err[jj] = np.linalg.norm(
            symmetric_tv_mode_product(
                A, s_x, m-1) - s_lbd * s_x)
        schur_time[jj] = t_end - t_start
        
        # now do rayleigh

        t_start = time.time()
        res_ray = rayleigh_quotient_iteration(
            e, x0, max_err=max_err,
            max_iter=max_itr, verbose=False,
            exit_by_diff=True)
        t_end = time.time()
        ray_time[jj] = t_end - t_start
        ray_cnt[jj] = res_ray['n_iter']
        ray_lbd[jj] = res_ray['lbd']
        ray_err[jj] = np.linalg.norm(res_ray['err'])
        # print("doing rayleigh")
        # print(res_ray)
        # print(e.L(res_ray['x'], res_ray['lbd']))

        # now do rayleigh chebyshev
        t_start = time.time()
        res_ray_cheb = rayleigh_chebyshev(
            e, x0, max_err=max_err, max_iter=max_itr,
            verbose=False, exit_by_diff=True)
        t_end = time.time()
        ray_cheb_time[jj] = t_end - t_start

        ray_cheb_cnt[jj] = res_ray_cheb['n_iter']
        ray_cheb_lbd[jj] = res_ray_cheb['lbd']
        ray_cheb_err[jj] = np.linalg.norm(res_ray_cheb['err'])
        # print("doing raychev")
        # print(res_ray_cheb)
        # print(e.L(res_ray_cheb['x'], res_ray_cheb['lbd']))

    summ = pd.DataFrame(
        {
            'o_ncm_iter': o_ncm_cnt,
            'schur_iter': schur_cnt,
            'ray_iter': ray_cnt, 'ray_cheb_iter': ray_cheb_cnt,
            'o_ncm_err': o_ncm_err,
            'schur_err': schur_err,
            'ray_err': ray_err, 'ray_cheb_err': ray_cheb_err,
            'o_ncm_lbd': o_ncm_lbd,
            'schur_lbd': schur_lbd,
            'ray_lbd': ray_lbd,
            'ray_cheb_lbd': ray_cheb_lbd,
            'o_ncm_time': o_ncm_time,
            'schur_time': schur_time,
            'ray_time': ray_time,
            'ray_cheb_time': ray_cheb_time
        },
        columns=['o_ncm_iter', 'o_ncm_lbd', 'o_ncm_err', 'o_ncm_time',
                 'schur_iter', 'schur_lbd', 'schur_err', 'schur_time',
                 'ray_iter', 'ray_lbd', 'ray_err', 'ray_time',
                 'ray_cheb_iter', 'ray_cheb_lbd', 'ray_cheb_err',
                 'ray_cheb_time'])
    return summ


We now run the routine for a small test size:

In [0]:
from IPython.display import display, HTML
np.random.seed(0)
k = 8
m = 4
max_err = 1e-10
max_itr = 200
n_test = 100

summ = test_eigen_tensor(k, m, max_err, max_itr, n_test)
# summ[['o_ncm_time', 'schur_time', 'ray_time', 'ray_cheb_time']].describe())
display(HTML(summ.describe().to_html()))



,o_ncm_iter,o_ncm_lbd,o_ncm_err,o_ncm_time,schur_iter,schur_lbd,schur_err,schur_time,ray_iter,ray_lbd,ray_err,ray_time,ray_cheb_iter,ray_cheb_lbd,ray_cheb_err,ray_cheb_time
count,100.000000,100.000000,1.000000e+02,100.000000,100.000000,100.000000,1.000000e+02,100.000000,100.000000,100.000000,1.000000e+02,100.000000,100.000000,100.000000,1.000000e+02,100.000000
mean,23.720000,0.335407,3.614095e-16,0.012036,21.400000,0.281598,4.407972e-16,0.006241,21.930000,0.380500,7.370778e-12,0.015353,27.820000,0.653453,5.477123e-12,0.029170
std,21.941189,1.308622,2.124016e-16,0.010758,14.567849,1.330551,2.103786e-16,0.004107,19.280626,1.336179,2.220565e-11,0.012497,21.969391,4.644036,2.909527e-11,0.022118
min,6.000000,-2.910113,9.928167e-17,0.003114,6.000000,-2.910113,1.727248e-16,0.001923,5.000000,-2.910113,1.358965e-16,0.004137,4.000000,-2.784721,9.574923e-17,0.005055
25%,12.000000,-0.287637,2.318626e-16,0.006179,12.000000,-0.360038,2.860289e-16,0.003600,11.000000,-0.287637,5.465922e-16,0.008510,10.000000,-1.191096,4.291264e-16,0.011046
50%,16.000000,0.306315,2.853492e-16,0.008070,16.000000,0.306315,3.821771e-16,0.004724,15.000000,0.306315,6.179607e-15,0.010620,23.000000,-0.216168,7.924395e-16,0.023993
75%,28.500000,1.004106,4.251793e-16,0.014196,28.500000,0.973710,5.390070e-16,0.008070,27.500000,1.028952,9.855966e-13,0.019893,40.000000,1.010268,9.071952e-15,0.040666
max,171.000000,3.437587,1.200890e-15,0.082043,83.000000,3.437587,1.430422e-15,0.023103,115.000000,3.437587,1.333120e-10,0.075155,136.000000,31.088427,2.677725e-10,0.128155


In [0]:
display(HTML(summ.head(10).to_html()))
        

,o_ncm_iter,o_ncm_lbd,o_ncm_err,o_ncm_time,schur_iter,schur_lbd,schur_err,schur_time,ray_iter,ray_lbd,ray_err,ray_time,ray_cheb_iter,ray_cheb_lbd,ray_cheb_err,ray_cheb_time
0,56,0.907212,3.960635e-16,0.036707,32,-0.358064,2.529665e-16,0.009139,30,-0.358064,2.665326e-14,0.021297,13,1.200351,1.020178e-10,0.014310
1,19,1.461345,2.593053e-16,0.009554,19,1.461345,7.344603e-16,0.005495,18,1.461345,3.138609e-13,0.012599,18,3.195664,2.873741e-15,0.018936
2,40,0.739221,2.742400e-16,0.019328,40,0.739221,3.340953e-16,0.011329,39,0.739221,3.074525e-16,0.026444,80,-0.107012,2.932345e-12,0.080181
3,33,0.739221,2.568563e-16,0.016515,36,0.739221,3.946029e-16,0.010091,33,0.739221,2.530702e-11,0.022220,8,-1.231871,1.140873e-15,0.010141
4,9,0.313733,3.210879e-16,0.004678,9,0.313733,2.486652e-16,0.002737,8,0.313733,3.963226e-16,0.007017,17,-1.508761,1.018724e-15,0.019475
5,12,-1.472034,3.628559e-16,0.006275,12,-1.472034,6.998163e-16,0.003507,11,-1.472034,5.478713e-16,0.013438,5,1.859172,6.330766e-16,0.006443
6,13,0.832607,5.302695e-16,0.006589,13,0.832607,5.163302e-16,0.003858,12,0.832607,3.572183e-16,0.008609,20,-0.107012,2.724682e-12,0.020889
7,18,0.465653,1.501112e-16,0.009157,18,0.465653,3.230873e-16,0.005234,17,0.465653,9.751338e-14,0.011873,30,-1.472034,5.788286e-16,0.031510
8,50,0.564058,3.213127e-16,0.027994,35,-0.213882,2.252270e-16,0.010073,41,-0.213882,5.627040e-14,0.028054,53,2.357049,6.653021e-16,0.055561
9,47,0.302301,3.097927e-16,0.023258,50,-0.107012,2.847127e-16,0.014058,52,3.195664,2.352520e-15,0.034827,11,-0.778608,1.881669e-11,0.012417


And bigger test size:

In [0]:
  np.random.seed(0)
  k = 15
  m = 4
  max_err = 1e-10
  max_itr = 200
  n_test = 500

  summ = test_eigen_tensor(k, m, max_err, max_itr, n_test)
  print(summ[[
      'o_ncm_time', 'schur_time', 'ray_time', 'ray_cheb_time']].describe())
      
        

       o_ncm_time  schur_time    ray_time  ray_cheb_time
count  500.000000  500.000000  500.000000     500.000000
mean     0.044818    0.028436    0.065076       0.214614
std      0.036399    0.022180    0.052277       0.086965
min      0.005064    0.003425    0.008043       0.008366
25%      0.015839    0.010127    0.024077       0.151355
50%      0.032340    0.021160    0.048661       0.269523
75%      0.062547    0.039411    0.085064       0.271158
max      0.158442    0.101542    0.237615       0.318366
